Global module containing constants and invariants

In [1]:
from fractions import Fraction
import sympy as sp
import numpy as np
from enum import Enum
from dataclasses import dataclass

In [2]:
x, y, z = sp.symbols('x y z')
OUTCOME_PROB = Fraction(1,6)

In [3]:
# XYZ studies f(x,y,z) - f(y,x,z)
# YZX studies f(y,z,x) - f(z,y,x)
# UUZ studies f(x,y,z) - f((x+y)/2, (x+y)/2, z)
# Thm 6: YYZ studies f(x,y,z) - f(y,y,z)
Version = Enum('Version', ['XYZ', 'YZX', 'UUZ', 'YYZ'])

# version specific info
@dataclass
class VersionSpecific:
    restriction: list
    pos_state: tuple
    neg_state: tuple
    data_dir: str

In [4]:
# For Version.UUZ
SCALE = 1000
Z_LB = 1/3

In [10]:
# version specific things
def version_info(ver):
    '''
    Retrieves info specific to the version.
    '''
    if ver == Version.XYZ:
        return VersionSpecific(
            restriction=[x>0, x<y, y<z],
            pos_state=(x,y,z),
            neg_state=(y,x,z),
            data_dir='xyz/'
        )
    elif ver == Version.YZX:
        return VersionSpecific(
            restriction=[z>0, z<x, x<y],
            pos_state=(y,z,x),
            neg_state=(z,y,x),
            data_dir='yzx/'
        )
    elif ver == Version.UUZ:
        return VersionSpecific(
            restriction=[x>0, x<y, x+y+z >= SCALE, x+y+z <= SCALE, z >= SCALE * Z_LB],
            pos_state=(x,y,z),
            neg_state=((x+y)/2,(x+y)/2,z),
            data_dir='uuz/'
        )
    elif ver == Version.YYZ:
        return VersionSpecific(
            restriction=[x>0, z>0, x<y],
            pos_state=(x,y,z),
            neg_state=(y,y,z),
            data_dir='yyz/'
        )
    else:
        print("Unknown version")
        exit(1)
            
def flip_state(s):
    ''' (x,y,z) -> (y,x,z) '''
    return (s[1], s[0], s[2])

def state_to_str(s):
    ''' Converts the state to a parenthised string. '''
    return f"({','.join(str(i) for i in s)})"

def cache_name(s):
    ''' Returns the cache name based on the given state. '''
    return 'h_n' + state_to_str(s)

def order(s,a,b,c):
    ''' Maps state in x,y,z to variables a,b,c, preserving the order. 
        E.g. (x,y,z) -> (a,b,c); (y,z,x) -> (b,c,a)
    '''
    var_map = {x:a, y:b, z:c}
    return tuple(var_map[i] for i in s)

In [6]:
# change this to control the version
VERSION = Version.YZX

V = version_info(VERSION).restriction
DATA_DIR = version_info(VERSION).data_dir
STATE = version_info(VERSION).pos_state
NEG_STATE = version_info(VERSION).neg_state
STATE_STR = state_to_str(STATE)
ORDER = (lambda a,b,c: order(STATE,a,b,c))

# output files
POS_H_CACHE = DATA_DIR + cache_name(STATE)
NEG_H_CACHE = DATA_DIR + cache_name(NEG_STATE)
COORDS_FILE = (lambda n: DATA_DIR + f'n={n}_coords_f')

F_MODEL_NAME = (lambda n: f'Delta{n}{STATE_STR}-alpha{n}')
DH_MODEL_NAME = (lambda n: f'h{n}{STATE_STR}-h{n}{state_to_str(NEG_STATE)}')
F_MIP_OUTPUT = (lambda n: DATA_DIR + F_MODEL_NAME + '.lp')
DH_MIP_OUTPUT = (lambda n: DATA_DIR + DH_MODEL_NAME + '.lp')

# plot f_n(·) > alpha_n
XYZ_SUM = 2000
CBAR_LABEL = (lambda n: f'∆{n}{STATE_STR}-(1/2)^{n}(4/5)')
FIG_TITLE = DATA_DIR + '{' + f'(x,y,z): 0<x<y<z, x+y+z={XYZ_SUM}, ∆n{STATE_STR} > (1/2)^n(4/5)' + '}'
FIG_NAME = DATA_DIR + f'∆n{STATE_STR}-alpha_n.png'

In [7]:
import collections
import functools

class memoized(object):
    '''Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    '''
    def __init__(self, func):
        self.func = func
        self.cache = {}
    
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            # uncacheable. a list, for instance.
            # better to not cache than blow up.
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    
    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__
    
    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

In [8]:
# @memoized
# def thresh(n):
#     '''
#     alpha_n = (1/2)^n (4/5)
#     '''
#     return Fraction(4, (2**n) * 5)

In [9]:
# Floating pt version of `thresh(n)`
# Faster with this version for intensive data handling
@memoized
def thresh(n):
    return 4/(2**n * 5)